In [17]:
import numpy as np
import pandas as pd

from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import classification_report

import os

In [18]:
import re
import spacy

In [19]:
df = pd.read_excel('Data/20181001-newindianexpress_sentence_classification_adjudicated_20181218.xlsx')

In [20]:
df = df[np.logical_not(np.isnan(np.array(df['label'])))]

In [21]:
for i, row in df.iterrows():
    row['sentence'] = re.sub("http\S*\s", "", row['sentence'])
    row['sentence'] = re.sub("((url)*\s*:\s*)*http\S*", "", row['sentence'])
    
    if row['sentence'] == "":
        df=df.drop([i])
        

In [22]:
df1_sents= df['sentence']
df1_y= df['label']

In [23]:
df2 = pd.read_excel('Data/20181126_NewIndianExpress_3Classifier0_Baglan-Ezgi-Balacan-Eylem-Merged_AgreedByAll.xlsx')
df2 = df2[np.logical_not(np.isnan(np.array(df2['label'])))]
df2 = df2[(np.array(df2['label'] == 0))]

In [25]:
nlp = spacy.load('en_core_web_lg')

In [26]:
df2_sents = []
df2_labels = []
for i, row in df2.iterrows():
    row['text'] = row['text'].replace('\n', ' ')
    row['text'] = row['text'].replace('&amp;', 'and')
    row['text'] = row['text'].replace('\xa0', ' ')
    row['text'] = row['text'].replace('-', '')
    row['text'] = row['text'].replace("\'", "'")
    row['text'] = re.sub("((url)*\s*:\s*)*http\S*", "", row['text'])
    row['text'] = row['text'].replace('  ', ' ')
    doc = nlp(row['text'])
    for sent in doc.sents:
        df2_sents.append(sent.text) 
        df2_labels.append(0)


### Stopwords

In [27]:
ordinals = ['1st', '2nd', '3rd', '4th']
for i in range(5,31):
    ordinals.append(str(i) + 'th')
    ordinals.append('0' + str(i) + 'th') 
    

In [28]:
number_stopwords = [str(i) for i in range(10001)] + ['0'+str(i) for i in range(100)] + ['000'] + ordinals

In [29]:
sents = df1_sents.tolist() + df2_sents
y = df1_y.tolist() + df2_labels

In [36]:
import pickle

In [37]:
data_path = 'Data/all_data.pickle'

In [38]:
Data_ = {}

In [39]:
Data_['X'] = np.array(sents)
Data_['y'] = np.array(y)
Data_['stopwords'] = np.array(number_stopwords)

In [40]:
with open(data_path, 'wb') as file_:
    pickle.dump(Data_, file_, protocol=pickle.HIGHEST_PROTOCOL)